In [1]:
import luigi
import pandas as pd
import numpy as np
import sys
sys.path.insert(0, '/home/fbuonerba/codes/')
from coinapi_v1 import CoinAPIv1
import datetime
from datetime import datetime, timedelta
import time
import calendar
import json
import urllib.request
import requests
import multiprocessing as mp

#test_key = 'DB318A59-25FF-499E-9A6D-783A19C346D8'

test_key = 'EED0F746-36FB-4CC4-8E7C-527333DFA6FB'

api = CoinAPIv1(test_key)

meta_path='/home/fbuonerba/codes/meta_data/'
with open(meta_path+'new_coins.txt') as f:
    coins=json.load(f)
# with open(meta_path+'symbols.txt') as ff:
#     symbols=json.load(ff)
quotes=['USD','BTC']

#assets = api.metadata_list_assets()
#symbols = api.metadata_list_symbols()
#exchanges = api.metadata_list_exchanges()

/usr/lib/python3.5/importlib/_bootstrap.py:222: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/usr/lib/python3.5/importlib/_bootstrap.py:222: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)


In [ ]:
###Following symbols apparently have no data available.
###random tests with request_ohlcv seem to confirm, i.e.
#it is not just a 'week' issue.

BTC-E_SPOT_BTC_USD []
BTC-E_SPOT_LTC_USD []
MTGOX_SPOT_BTC_USD []
LAKEBTC_SPOT_BTC_USD []
BITTREX_SPOT_BTS_BTC []
THEROCKTRADING_SPOT_BTC_USD []
BTCX_SPOT_BTC_USD []
BITVC_SPOT_BTC_BTC []
QUOINE_SPOT_XRP_BTC []
QUOINE_SPOT_LTC_BTC []
QUOINE_SPOT_TRX_BTC []
QUOINE_SPOT_QTUM_BTC []
QUOINE_SPOT_ETC_BTC []
QUOINE_SPOT_XEM_BTC []
QUOINE_SPOT_XMR_BTC []
QUOINE_SPOT_ZEC_BTC []
QUOINE_SPOT_DASH_BTC []
QUOINE_SPOT_NEO_BTC []
CRYPTOPIA_SPOT_WAVES_BTC []
CRYPTOPIA_SPOT_BCN_BTC []
QUOINE_SPOT_BCH_BTC []
CRYPTOPIA_SPOT_SC_BTC []
CCEX_SPOT_XVG_BTC []
BTER_SPOT_QTUM_BTC []
BTER_SPOT_DOGE_BTC []
BTER_SPOT_BCH_BTC []
THEROCKTRADING_SPOT_LTC_USD []
QUOINE_SPOT_XLM_BTC []
BTER_SPOT_LTC_BTC []
LIQUI_SPOT_QTUM_BTC []
BTER_SPOT_ZEC_BTC []
BTER_SPOT_ETH_BTC []
QUOINE_SPOT_DASH_USD []
BTER_SPOT_XRP_BTC []
CCEX_SPOT_XVG_USD []
BTER_SPOT_ETC_BTC []
CCEX_SPOT_BTG_BTC []
BTER_SPOT_EOS_BTC []
BTER_SPOT_BAT_BTC []
BTER_SPOT_OMG_BTC []
BTER_SPOT_BTM_BTC []
QUOINE_SPOT_VET_BTC []
BTER_SPOT_ZRX_BTC []
BTER_SPOT_XEM_BTC []
BTER_SPOT_REP_BTC []
BTER_SPOT_XMR_BTC []
BTER_SPOT_DOGE_USD []
TIDEX_SPOT_STEEM_BTC []
CCEX_SPOT_BTG_USD []
TIDEX_SPOT_BTS_BTC []
LIQUI_SPOT_STEEM_BTC []
BITBAY_SPOT_ETH_USD []
BITBAY_SPOT_LSK_USD []
BITBAY_SPOT_BTC_USD []
BITBAY_SPOT_LTC_USD []
ACX_SPOT_BTC_USD []
BITBAY_SPOT_DASH_USD []
BITBAY_SPOT_LSK_BTC []
BITBAY_SPOT_DASH_BTC []
BITBAY_SPOT_LTC_BTC []
BITBAY_SPOT_BCH_BTC []
BITBAY_SPOT_BTG_USD []
BITBAY_SPOT_BCH_USD []
BITBAY_SPOT_BTG_BTC []
BITBAY_SPOT_ETH_BTC []
XBTCE_SPOT_BTC_USD []
BTCTURK_SPOT_ETH_BTC []
XBTCE_SPOT_ETH_USD []
XBTCE_SPOT_LTC_USD []
QUOINE_SPOT_REP_BTC []
XBTCE_SPOT_LTC_BTC []
XBTCE_SPOT_BCH_BTC []
XBTCE_SPOT_BCH_USD []
XBTCE_SPOT_XRP_USD []
XBTCE_SPOT_ETH_BTC []
QUOINE_SPOT_GNT_BTC []
XBTCE_SPOT_ETC_BTC []
QUOINE_SPOT_OMG_BTC []
XBTCE_SPOT_EOS_BTC []
XBTCE_SPOT_XRP_BTC []
XBTCE_SPOT_XMR_BTC []
XBTCE_SPOT_NEO_BTC []
LYKKE_SPOT_ZRX_BTC []
ANXPRO_SPOT_XRP_BTC []
XBTCE_SPOT_IOTA_BTC []
COINEXCHANGE_SPOT_LTC_BTC []
ANXPRO_SPOT_BTC_USD []
ANXPRO_SPOT_LTC_BTC []
ANXPRO_SPOT_DOGE_BTC []
LYKKE_SPOT_BTC_USD []
LYKKE_SPOT_PPT_BTC []
COINEXCHANGE_SPOT_DASH_BTC []
COINEXCHANGE_SPOT_DOGE_BTC []
COINEXCHANGE_SPOT_DGB_BTC []
COINEXCHANGE_SPOT_BCH_BTC []
COINEXCHANGE_SPOT_QTUM_BTC []
COINEXCHANGE_SPOT_ETC_BTC []
COINEXCHANGE_SPOT_ZEC_BTC []
LYKKE_SPOT_AE_BTC []
COINEXCHANGE_SPOT_ETH_BTC []
LYKKE_SPOT_LTC_USD []
LYKKE_SPOT_LTC_BTC []
LYKKE_SPOT_OMG_BTC []
LYKKE_SPOT_EOS_BTC []
LYKKE_SPOT_GNT_BTC []
LYKKE_SPOT_REP_BTC []
ABUCOINS_SPOT_ZIL_BTC []
BITCOINID_SPOT_DASH_BTC []
BITCOINID_SPOT_XLM_BTC []
BITCOINID_SPOT_XEM_BTC []
XBTCE_SPOT_LSK_BTC []
XBTCE_SPOT_OMG_BTC []
XBTCE_SPOT_XLM_BTC []
XBTCE_SPOT_XEM_BTC []
XBTCE_SPOT_ADA_BTC []
XBTCE_SPOT_REP_BTC []
XBTCE_SPOT_ZEC_BTC []
LYKKE_SPOT_BCH_BTC []
LYKKE_SPOT_BCH_USD []
LYKKE_SPOT_DASH_USD []
LYKKE_SPOT_DASH_BTC []
LYKKE_SPOT_ETC_BTC []
LYKKE_SPOT_ETC_USD []
LYKKE_SPOT_ETH_BTC []
LYKKE_SPOT_ETH_USD []
LYKKE_SPOT_ICX_BTC []
COINEXCHANGE_SPOT_EOS_BTC []
LYKKE_SPOT_ZEC_BTC []
LYKKE_SPOT_ZEC_USD []
COINEXCHANGE_SPOT_TRX_BTC []
COINEXCHANGE_SPOT_OMG_BTC []
COINEXCHANGE_SPOT_XVG_BTC []
COINEXCHANGE_SPOT_GNT_BTC []
COINEXCHANGE_SPOT_PPT_BTC []
NOVA_SPOT_DCR_BTC []
BLEUTRADE_SPOT_BTC_BTC []
LYKKE_SPOT_BTG_USD []
LYKKE_SPOT_BTG_BTC []
BITBAY_SPOT_REP_USD []


In [ ]:
#Getting data from CoinMarketCap#
web_data= urllib.request.urlopen('https://api.coinmarketcap.com/v2/ticker/')
cmc_data=web_data.read()
cmc=json.loads(cmc_data.decode('utf-8'))
cmc=cmc['data']
xxx=list(cmc.values())
L=[]
for x in xxx:
    L.append(x['rank'])
L=np.array(L)
xdict={}
for x in xxx:
    if x['rank']<51:
        xdict[x['rank']]=x['symbol']


In [ ]:
COINS=list(xdict.values())
new_coins=[]
C=np.array(COINS)
C[10]='IOTA'
#delete tether and mysterious MOAC...
C1=np.delete(C, [39])
C2=np.delete(C1, [7])
with open('/home/fbuonerba/codes/meta_data/new_coins.txt','w') as file:
    json.dump(list(C2),file)
C2

In [ ]:
symbols
new_symbols=[]

#remove Bitmex, it is outlier.
for x in symbols:
    if (x['asset_id_base'] in C2) and (x['asset_id_quote'] in quotes) and (x['symbol_type']=='SPOT') and (x['exchange_id']!='BITMEX'):
        new_symbols.append(x['symbol_id'])
        
with open('/home/fbuonerba/codes/meta_data/new_symbols.txt','w') as file:
    json.dump(new_symbols,file)
    


In [ ]:
new_cmc_id={}
for x in xxx:
    if (x['rank']<=50) and (x['symbol']!='MOAC') and (x['symbol']!='USDT'):
        new_cmc_id[x['id']]=x['symbol']
new_cmc_id
with open('/home/fbuonerba/codes/meta_data/new_cmc_id.txt','w') as file:
    json.dump(new_cmc_id,file)
len(new_cmc_id)

In [ ]:
#Getting id of coins listed on CoinMarketCap which appear in our top_coins list.
#start=100*n+1, since they limit to 100 coins per request.
web_data1= urllib.request.urlopen('https://api.coinmarketcap.com/v2/ticker/?start=1701')
cmc_data1=web_data1.read()
cmc1=json.loads(cmc_data1.decode('utf-8'))
cmc1=cmc1['data']
co1=0
xxx1=list(cmc1.values())
for x in xxx1:
    if x['symbol'] in coins:
        print(x['symbol'])
        diction[x['id']]=x['symbol']
        co1+=1
print(co1, diction, len(diction))

In [ ]:
with open(meta_path+'symbols.txt') as ll:
    symbols=json.load(ll)

In [ ]:
import os
count=0
gcount=0
S=os.listdir('/home/fbuonerba/ohlcv_data')
for s in S:
    for sym in symbols:
        if sym in s:
            gcount+=1
            with open('/home/fbuonerba/ohlcv_data/' + s,'r') as file:
                data=file.read()
            if data=='{}':
                count+=1
                print(count/gcount)
                

In [ ]:
web_data= urllib.request.urlopen('https://coinmarketcap.com/historical/20180701/')
cmc_data=web_data.read()#.decode('utf-8')

In [ ]:
cmc_data

In [ ]:
headers = {'Accept': 'text/html'}
url='https://coinmarketcap.com/historical/20180701/'
web_data= requests.get(url, headers=headers)
cmc_data=web_data.content

In [ ]:
from bs4 import BeautifulSoup
soup=BeautifulSoup(cmc_data, 'html.parser')

In [ ]:
soup

In [ ]:
So=soup.find_all('td')
So


In [ ]:
test=[]
for x in So:
    #print(x.contents)
    if x.contents[0]=='\n':
        z=str(x.contents[1])
        if 'data-supply' in z:
            y=z.split('=')[1].split('data')[0].split('"')[1]
            print(y=='None')
            test.append(y)
        


In [ ]:
Vol=[]
SoVolume=soup.find_all(class_='volume')
for x in SoVolume:
    x=x.contents[0].replace('$','').replace(',','')
    x=x.replace('Low Vol','0')
    x=int(float(x))
    Vol.append(x)


In [ ]:
Supply_span=[]
Supply_a=[]
SoSupply_span=soup.find_all('span',{'data-supply':True})
SoSupply_a=soup.find_all('a',{'data-supply':True})
#SoSupply=soup.find_all('a',{'data-supply':True})
print(len(test), len(SoVolume))

for x in SoSupply_a:
    x=x.contents[0].strip('\n').replace(',','')
    Supply_a.append(x)
for x in SoSupply_span:
    x=x.contents[0].strip('\n').replace(',','')
    Supply_span.append(x)
print(Supply_a)
print('+++++++++++++++++++')
print(Supply_span)


In [ ]:
Price=[]
SoPrice=soup.find_all(class_='price')
for x in SoPrice:
    x=x.contents[0].replace('$','').replace(',','')
    x=float(x)
    Price.append(x)
Price

In [ ]:
Syms=[]
SoSym=soup.find_all(class_="text-left col-symbol")
for x in SoSym:
    x=str(x.contents[0])
    Syms.append(x)



In [ ]:
data={}
for i in range(len(Supply)):
    data[Syms[i]]={'volume': Vol[i], 'price': Price[i], 'supply': Supply[i]}

In [ ]:
json_data=json.dumps(data)

In [ ]:
with open('/home/fbuonerba/codes/meta_data/new_symbols.txt') as ff:
    syms=json.load(ff)
len(syms)

In [ ]:
import sys
import numpy as np
sys.path.insert(0, '/home/fbuonerba/codes/')
from mp_functions import compute_log_marketcap as lmkcap
lmkcap('MKR','USD', 1515283200,1525132800)

In [ ]:
import sys
import numpy as np
import json
sys.path.insert(0, '/home/fbuonerba/codes/')
from mp_functions import compute_log_marketcap as clm

with open('/home/fbuonerba/codes/meta_data/top_coins.txt') as ff:
    coins=json.load(ff)
clm('EUR','USD',1515283200,1522540800)
for coin in coins:
    x=clm(str(coin),'USD',1515283200,1522540800)
    print(coin, x)